In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

## 1、生成数据集 

In [2]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
#根据给定权重和偏置，生成一个数据集
features,labels = d2l.synthetic_data(true_w,true_b,1000)

## 2、读取数据集 

In [3]:
batch_size=10
#data模块可以生成数据集
dataset = data.TensorDataset(*(features,labels))
#利用数据集生成随机小批量数据集的生成器
data_iter = data.DataLoader(dataset,batch_size,shuffle=True)

In [4]:
next(iter(data_iter))

[tensor([[ 0.5034,  1.3254],
         [-1.1126, -0.1871],
         [ 0.4183, -0.8816],
         [-0.5122,  1.5305],
         [ 0.1538, -1.0363],
         [-0.9922,  1.4266],
         [-0.9214, -0.8309],
         [ 0.0037,  0.0580],
         [ 0.8835,  0.3094],
         [ 0.0361, -0.5365]]),
 tensor([[ 0.7055],
         [ 2.6162],
         [ 8.0257],
         [-2.0380],
         [ 8.0203],
         [-2.6388],
         [ 5.1925],
         [ 4.0178],
         [ 4.9164],
         [ 6.1172]])]

## 3、定义模型 

In [5]:
from torch import nn
#只有一个全连接层的深度学习模型
net = nn.Sequential(nn.Linear(2,1))

## 4、初始化参数 

In [6]:
#初始化模型的第一层参数，但只有一个层
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 5、定义损失函数 

In [7]:
#这就是均方误差的损失函数
loss = nn.MSELoss()

## 6、定义优化算法 

In [8]:
[item for item in net.parameters()]

[Parameter containing:
 tensor([[-0.0012,  0.0014]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True)]

In [9]:
# 小批量随机梯度下降法
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

## 7、训练 

In [10]:
num_epochs= 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        # 1、计算小批量总损失，前向传播。
        l = loss(net(X),y)
        #避免梯度叠加
        trainer.zero_grad()
        #2、计算损失函数的梯度
        l.backward()
        #3、利用学习算法，优化参数
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch+1},loss {l:f}')    

epoch 1,loss 0.000308
epoch 2,loss 0.000099
epoch 3,loss 0.000099


In [11]:
w = net[0].weight.data
print('w的估计误差',true_w-w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：',true_b-b)

w的估计误差 tensor([-0.0003,  0.0004])
b的估计误差： tensor([7.1526e-06])


## 练习 

In [12]:
# huber损失对应Pytorch的SmoothL1损失
loss = nn.SmoothL1Loss(beta=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        #开始计算梯度
        trainer.zero_grad()
        l.backward()
        trainer.step()  #对所有参数进行更新
    print("epoch: {}, loss:{}".format(epoch + 1, l))

epoch: 1, loss:0.00010626004223013297
epoch: 2, loss:3.3190397516591474e-05
epoch: 3, loss:0.00014073392958380282


In [13]:
net[0].weight.grad,net[0].bias.grad

(tensor([[0.0004, 0.0034]]), tensor([0.0052]))

In [14]:
net[0].weight

Parameter containing:
tensor([[ 2.0000, -3.4007]], requires_grad=True)